<a href="https://colab.research.google.com/github/shivanshmishr/Resume-Matching-with-Job-Descriptions/blob/main/Resume_dataset_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

# Step 1: Load the CSV Dataset
csv_file_path = "Resume.csv"  # Replace with the actual file path to your CSV dataset
df = pd.read_csv(csv_file_path)

# Step 2: Preprocess the CSV Dataset
# Assuming the CSV has a column named "resume_text" containing the text content of resumes
if "Resume_str" in df.columns:
    resumes = df["Resume_str"].tolist()
    print(f"Loaded {len(resumes)} resumes from the CSV dataset.")
else:
    print("The CSV dataset does not contain a 'resume_text' column. Please adjust column name as needed.")



Loaded 2484 resumes from the CSV dataset.


In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.6 MB/s eta 0:00:00


In [8]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import pandas as pd

# Load the CSV Dataset (Assuming you've already loaded it, as shown in Step 1)
csv_file_path = "Resume.csv"
df = pd.read_csv(csv_file_path)
if "Resume_str" in df.columns:
    resumes = df["Resume_str"].tolist()
else:
    print("The CSV dataset does not contain a 'Resume_str' column. Please adjust column name as needed.")

# List of Job Descriptions (You can replace this with your job descriptions)
job_descriptions = [
    "Software Engineer with expertise in Python and Django.",
    "Data Scientist with a background in machine learning.",
"Software Engineer with expertise in Python and Django. Experience in building web applications and APIs.",
"Data Scientist with a background in machine learning and statistical analysis. Proficient in Python and data visualization.",
"Marketing Manager responsible for developing and executing marketing strategies. Skilled in market research and campaign management.",
"Graphic Designer experienced in Adobe Creative Suite. Proficient in creating visuals for digital and print media.",
"Financial Analyst with expertise in financial modeling and data analysis. Proficient in Excel and financial software.",
"Customer Support Representative with strong communication skills. Experienced in handling customer inquiries and resolving issues.",
"Product Manager with experience in product development and launch. Skilled in market research and product lifecycle management.",
"UX/UI Designer with a focus on user-centered design principles. Proficient in wireframing and prototyping tools.",
"Sales Representative with a track record of exceeding sales targets. Strong interpersonal and negotiation skills.",
"HR Manager responsible for recruitment and employee relations. Experienced in HR policies and compliance.",

]

# Initialize DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

# Initialize lists to store embeddings
resume_embeddings = []
job_description_embeddings = []

# Tokenize and obtain embeddings for resumes
for Resume_str in resumes:
    input_ids = tokenizer.encode(Resume_str, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        embeddings = model(input_ids).last_hidden_state.mean(dim=1)
    resume_embeddings.append(embeddings)

# Tokenize and obtain embeddings for job descriptions
for job_description_text in job_descriptions:
    input_ids = tokenizer.encode(job_description_text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        embeddings = model(input_ids).last_hidden_state.mean(dim=1)
    job_description_embeddings.append(embeddings)

# Convert the lists of embeddings to tensors
resume_embeddings = torch.cat(resume_embeddings)
job_description_embeddings = torch.cat(job_description_embeddings)

# Now, you have the embeddings for resumes (resume_embeddings) and job descriptions (job_description_embeddings).


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between job descriptions and resumes
similarity_matrix = cosine_similarity(job_description_embeddings, resume_embeddings)

# Now, the similarity_matrix contains cosine similarity scores for each job description-resume pair.
# Each row corresponds to a job description, and each column corresponds to a resume.


In [10]:
# Specify the number of top candidates you want to retrieve for each job description
top_candidates_count = 5  # You can adjust this as needed

# Initialize a list to store the top candidates for each job description
top_candidates = []

# Loop through each job description to find the top candidates
for i, job_description_text in enumerate(job_descriptions):
    # Get the similarity scores for the current job description
    similarity_scores = similarity_matrix[i]

    # Get the indices that would sort the scores in descending order
    sorted_indices = (-similarity_scores).argsort()

    # Retrieve the top candidates based on the sorted indices
    top_candidates_for_job = [(j, similarity_scores[j]) for j in sorted_indices[:top_candidates_count]]

    # Append the top candidates to the list
    top_candidates.append((job_description_text, top_candidates_for_job))

# Now, 'top_candidates' contains the top candidates for each job description, along with their similarity scores.


In [11]:
for i, job_description_text in enumerate(job_descriptions):
    # ... (rest of the code)
    print(f"Job Description: {job_description_text}")
    print("Top Candidates:")
    for candidate_index, similarity_score in top_candidates_for_job:
        print(f"Resume Index: {candidate_index}, Similarity Score: {similarity_score}")

Job Description: Software Engineer with expertise in Python and Django.
Top Candidates:
Resume Index: 7, Similarity Score: 0.8547621965408325
Resume Index: 1606, Similarity Score: 0.8441619277000427
Resume Index: 4, Similarity Score: 0.8433263301849365
Resume Index: 12, Similarity Score: 0.8427091240882874
Resume Index: 83, Similarity Score: 0.8422747850418091
Job Description: Data Scientist with a background in machine learning.
Top Candidates:
Resume Index: 7, Similarity Score: 0.8547621965408325
Resume Index: 1606, Similarity Score: 0.8441619277000427
Resume Index: 4, Similarity Score: 0.8433263301849365
Resume Index: 12, Similarity Score: 0.8427091240882874
Resume Index: 83, Similarity Score: 0.8422747850418091
Job Description: Software Engineer with expertise in Python and Django. Experience in building web applications and APIs.
Top Candidates:
Resume Index: 7, Similarity Score: 0.8547621965408325
Resume Index: 1606, Similarity Score: 0.8441619277000427
Resume Index: 4, Similarit

In [12]:
print("Job Descriptions:")
print(job_descriptions)

print("Similarity Matrix:")
print(similarity_matrix)

print("Top Candidates:")
print(top_candidates)


Job Descriptions:
['Software Engineer with expertise in Python and Django.', 'Data Scientist with a background in machine learning.', 'Software Engineer with expertise in Python and Django. Experience in building web applications and APIs.', 'Data Scientist with a background in machine learning and statistical analysis. Proficient in Python and data visualization.', 'Marketing Manager responsible for developing and executing marketing strategies. Skilled in market research and campaign management.', 'Graphic Designer experienced in Adobe Creative Suite. Proficient in creating visuals for digital and print media.', 'Financial Analyst with expertise in financial modeling and data analysis. Proficient in Excel and financial software.', 'Customer Support Representative with strong communication skills. Experienced in handling customer inquiries and resolving issues.', 'Product Manager with experience in product development and launch. Skilled in market research and product lifecycle manage